<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [173]:
import librosa
import os
from scipy import signal 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
def downsampling(data,num):
  '''
   Downsampling por num(int) feito como se fosse no C
  '''
  d = np.copy(data)
  j=0
  for i in range(len(d)):

    if i%int(num) == 0:
      j +=1
    else:
      d[i] = 0.0
  datadown = np.zeros(j)
  j=0
  for i in range(len(d)):
  
    if d[i] != 0.0: 
      #print(d[i])
      datadown[j] = d[i]
      j += 1  

  return datadown

def amostragem(data, fs, plot = False):
  '''
   Amostragem de downsampling de uma serie temporal segundo Nyquist-Shannon. Alem disso o numero maximo
   de frequencias com o spectro calculado por ciclo deve ser igual ao numero de pontos N/2 (principio de incerteza).

   
   argumentos:
   
   - data: serie temporal
   - fs: frequência de amostragem do encode original
   - plot: opcional, plota o espectrograma com parametros padrões para obter máxima frequência maior que a largura de banda.
  '''
 
  f,t,Sxx = signal.stft(data,fs,nfft=fs/4,nperseg=fs/5,noverlap=fs/10)
  Sxx = np.abs(Sxx)
  freq = np.array([i for i in zip(t,Sxx.T) if np.shape(f[i[1]>np.std(i[1])])[0] != 0])

  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13) )
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

  return [[ i[0], np.ceil(np.float(fs)/(2.0*np.max(f[i[1]>np.std(i[1])]) )) ] for i in freq]

def subbed_timeseries(measure,fs):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=False))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )

  return a,fsa

def subbed_spect(measure,fs,plot=True):
  '''
   Return Sxx (log squared amplitude of stft spectra), f (frequencies), t (time steps)
  '''
  sampa = np.min(np.array(amostragem(np.array(measure),fs, plot=plot))[:,1])  
  fsa = (fs/int(sampa))
    
  a = downsampling(np.array(measure),sampa )
  f,t,Sxx = signal.stft(a,fsa,nfft=fsa/4,nperseg=fsa/5,noverlap=fsa/10)
  Sxx = np.abs(Sxx)
  #Cut zero variance part of spectra(useless recording)   
  Sxx = Sxx[:,np.var(np.log(Sxx+1e-13),axis=0) > 0.5]
  l = len(Sxx[0,:])
  t = t[:l] 
  
  if plot:
    plt.figure()
    plt.pcolormesh(t, f, np.log(Sxx+1e-13))
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    
  return np.abs(np.log(Sxx+1e-13)),f,t

In [172]:
# Load Semantic data From UrbanSound8K dataset, will be used as semantic noise.
df = pd.read_csv('/home/stattus4dpenalva/stattus4/mock_data/urban/UrbanSound8K/metadata/UrbanSound8K.csv',sep=',')
'''
0 = air_conditioner
1 = car_horn
2 = children_playing
3 = dog_bark
4 = drilling
5 = engine_idling
'''

group_class = df.iloc[df.groupby('fold').indices[1]]
group_class = group_class.set_index(["class"])
names = group_class.loc[ [ "air_conditioner","car_horn","children_playing","dog_bark","engine_idling" ],["slice_file_name"] ]

mockraw_list = []
mockspec_list = []
for a in names.values:  
  data,fs = librosa.load('/home/stattus4dpenalva/stattus4/mock_data/urban/UrbanSound8K/audio/fold1/'+a[0],sr=None)
  mockraw_list.append( (data,fs) )
  #mockspec_list.append( subbed_spect(data,fs) )
#print(df.iloc[group_class.indices[i][:11]])

In [245]:
# Adding two time-series for mocking semantic noise
nomesref = [f for f in os.listdir('/home/stattus4dpenalva/stattus4/wav_data_2018/')]
raw_list = []
spec_list = []
for i in range(len(nomesref[0:100])):
  #ignore .mat
  
  data,fs = librosa.load('/home/stattus4dpenalva/stattus4/wav_data_2018/'+nomesref[i],sr=None)
  raw_list.append( (data,fs) )
  #spec_list.append( subbed_spect(data,fs) )   
  l0 = len(raw_list[i][0][:])
  fs = raw_list[i][1]
  for j in range(250,400):
    l1 = len(mockraw_list[j][0][:])
    l1r = np.random.randint(1000,high=l1,size=1)[0]
    if l0 > l1:

      a = np.zeros(l0)
      
      stmock = np.std(mockraw_list[j][0][:]) +1e-8
      l  = l0 - l1r          
        
      straw = np.std(raw_list[i][0][:l1r])+1e-8
      straw2 = np.std(raw_list[i][0][l:])+1e-8
      straw1 = np.std(raw_list[i][0][l1r:l+l1r])+1e-8                  
        
      a[:l1r] = raw_list[i][0][:l1r]/(2.*straw)
      a[l1r:l+l1r] = raw_list[i][0][l1r:l+l1r]/(2.*straw1)    
      
        
      a[:l1r] = (mockraw_list[j][0][:l1r]/(2.0*stmock)) + a[:l1r]    
    
      if l > l1r:
        
        stmock1 = np.std(mockraw_list[j][0][:l1r]) +1e-8
        a[l1r:2*l1r] = (mockraw_list[j][0][:l1r]/(2.0*stmock1)) + a[l1r:2*l1r]    
      else:
        stmock1 = np.std(mockraw_list[j][0][:l]) +1e-8
        a[l1r:l1r+l] = (mockraw_list[j][0][:l]/(2.0*stmock1)) + a[l1r:l1r+l]    

    librosa.output.write_wav('/home/stattus4dpenalva/stattus4/mock_data/gen/'+nomesref[i]+'{}'.format(j)+'gen.wav',a,fs)

In [258]:
# Pareamento com vazamento, sem vazamento
nomesref = [f for f in os.listdir('/home/stattus4dpenalva/stattus4/wav_data_2018/')]
sv=[]
cv=[]
for f in nomesref:
  if f[0:2] == 'SV':
    data,fs = librosa.load('/home/stattus4dpenalva/stattus4/wav_data_2018/'+f,sr=None)
    sv.append(data)
  if f[0:2] == 'CV':
    data,fs = librosa.load('/home/stattus4dpenalva/stattus4/wav_data_2018/'+f,sr=None)
    cv.append(data)
    
for i in range(len(cv)):
  lcv = len(cv[i])
  lsv = len(sv[i])
  
  if lsv > lcv:
    esv = np.sum( np.square(sv[i][:lcv]))
    ecv = np.sum( np.square(cv[i][:lcv]))
    a = (0.8*cv[i][:lcv]*ecv+0.2*sv[i][:lcv]*esv)/(esv+ecv)
  else:
    esv = np.sum( np.square(sv[i][:lsv]))
    ecv = np.sum( np.square(cv[i][:lsv]))
    a = (0.8*cv[i][:lsv]*ecv+0.2*sv[i][:lsv]*esv)/(esv+ecv)
  librosa.output.write_wav('/home/stattus4dpenalva/stattus4/mock_data/gen/mock{}'.format(i)+'.wav',a,fs)